In [92]:
# Imports
import numpy as np          # Importerar numpy-biblioteket, ett populärt paket för vetenskapliga beräkningar och arbete med stora flerdimensionella matriser.
import random               # Importerar random-modulen, som tillhandahåller funktioner för att generera slumpmässiga tal och val.
from datetime import datetime # Importerar datetime-klassen från datetime-modulen, används för att arbeta med datum och tid.


In [93]:
# Parameters
population_sizes = [10, 20, 50, 100]  # Olika populationsstorlekar att testa
mutation_rates = [0.9, 0.6, 0.3, 0.1]  # Olika mutationsfrekvenser att testa

n_cities = 20            # Definierar antalet städer i problemet, troligen för ett resande säljare-problem (TSP) eller liknande optimeringsproblem.


In [94]:
# Generating a list of coordinates representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities), np.random.randint(0,100,n_cities))]
# Skapar en lista med koordinater för varje stad. Koordinaterna är slumpmässigt genererade med heltal mellan 0 och 100, både för x- och y-värden. 
# Listförståelse används här för att skapa en lista med par [x, y] för varje stad.

names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 
                       'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
# Skapar en NumPy-array som innehåller namn på 20 städer. Varje namn motsvarar en stad.

cities_dict = { x:y for x,y in zip(names_list, coordinates_list)}
# Skapar en dictionary (ordbok) där varje stadsnamn (från names_list) mappas till en uppsättning koordinater (från coordinates_list). 
# Detta gör det enkelt att slå upp en stads koordinater baserat på namnet.

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5
# Denna funktion beräknar avståndet mellan två punkter (a och b) med hjälp av Euklidiskt avstånd. 
# a och b är listor som innehåller två värden (x, y), och formeln används för att beräkna distansen mellan dessa punkter.

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])
# Denna funktion tar två städers namn som argument (city_a och city_b) och använder cities_dict för att slå upp deras koordinater.
# Sedan används funktionen compute_city_distance_coordinates för att beräkna avståndet mellan städerna baserat på deras koordinater.

cities_dict
# Returnerar ordboken (cities_dict) så att du kan se eller använda den i senare kod.


{'Berlin': [10, 82],
 'London': [2, 80],
 'Moscow': [97, 78],
 'Barcelona': [52, 22],
 'Rome': [32, 27],
 'Paris': [59, 21],
 'Vienna': [13, 79],
 'Munich': [37, 87],
 'Istanbul': [67, 96],
 'Kyiv': [78, 63],
 'Bucharest': [0, 14],
 'Minsk': [94, 35],
 'Warsaw': [59, 63],
 'Budapest': [69, 38],
 'Milan': [41, 9],
 'Prague': [42, 14],
 'Sofia': [54, 65],
 'Birmingham': [13, 40],
 'Brussels': [80, 84],
 'Amsterdam': [79, 15]}

In [95]:
# First step: Create the first population set
def genesis(city_list, n_population):
    # Skapar en funktion som genererar den initiala populationen av lösningar.
    # city_list är en lista över alla städer, och n_population är antalet lösningar som ska genereras.

    population_set = []  # Skapar en tom lista för att lagra varje genererad lösning.
    for i in range(n_population):  # Loopar n_population gånger för att generera varje lösning.
        # Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        # Genererar en slumpmässig lösning genom att slumpmässigt välja n_cities städer från city_list utan återplacering.
        # np.random.choice används för att slumpmässigt välja index och sedan hämta motsvarande städer.

        population_set.append(sol_i)  # Lägger till den genererade lösningen till population_set-listan.

    return np.array(population_set)  # Returnerar populationen som en NumPy-array för mer effektiv hantering.

population_set = genesis(names_list, n_population)  # Genererar den första populationen av lösningar.
population_set  # Visar populationen.


array([['Sofia', 'Munich', 'Rome', ..., 'Kyiv', 'Istanbul', 'London'],
       ['Amsterdam', 'Barcelona', 'Berlin', ..., 'Paris', 'Kyiv',
        'Warsaw'],
       ['Istanbul', 'Munich', 'Paris', ..., 'Vienna', 'Prague',
        'Birmingham'],
       ...,
       ['Birmingham', 'Bucharest', 'Berlin', ..., 'Munich', 'Prague',
        'Paris'],
       ['Vienna', 'Milan', 'Prague', ..., 'Istanbul', 'Sofia', 'Rome'],
       ['Sofia', 'Paris', 'Brussels', ..., 'Berlin', 'Milan',
        'Amsterdam']], dtype='<U10')

In [96]:
def fitness_eval(city_list, cities_dict):
    # Denna funktion beräknar fitness-värdet (eller kostnaden) för en viss lösning (city_list) genom att summera avstånden mellan alla städer.
    # city_list är en lista över städer i den ordning de besöks, och cities_dict är en ordbok med stadens namn som nyckel och koordinater som värde.

    total = 0  # Initialiserar totalavståndet till noll.
    for i in range(n_cities - 1):  # Loopar genom varje stad i lösningen fram till den näst sista staden.
        a = city_list[i]  # Hämtar den aktuella staden.
        b = city_list[i + 1]  # Hämtar nästa stad i listan.
        total += compute_city_distance_names(a, b, cities_dict)  
        # Beräknar avståndet mellan den aktuella staden (a) och nästa stad (b), 
        # och lägger till detta avstånd till totalvärdet.

    return total  # Returnerar den totala distansen för staden i den aktuella ordningen.


In [97]:
def get_all_fitnes(population_set, cities_dict):
    # Denna funktion beräknar fitness-värdet för varje lösning i hela populationen.
    # population_set är en uppsättning av alla lösningar, och cities_dict är en ordbok med städers namn och koordinater.

    fitnes_list = np.zeros(n_population)
    # Skapar en NumPy-array fylld med nollor som kommer att lagra fitness-värdet för varje lösning. Arrayen har storleken n_population.

    # Looping over all solutions computing the fitness for each solution
    for i in range(n_population):  # Loopar genom varje lösning i populationen.
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)
        # Beräknar fitness-värdet för den i:te lösningen genom att använda fitness_eval-funktionen och lagrar resultatet i fitnes_list.

    return fitnes_list  # Returnerar listan med fitness-värden.

fitnes_list = get_all_fitnes(population_set, cities_dict)  # Anropar funktionen för att beräkna fitness för hela populationen.
fitnes_list  # Visar listan med fitness-värden.


array([1051.46749426,  935.8248257 , 1172.90652377, 1091.31447528,
        980.81657555, 1167.36375564, 1038.95089306, 1087.16180889,
       1133.81593114, 1052.83512667, 1149.20402089,  937.08065763,
       1095.28764842, 1006.43608781, 1007.18144838, 1145.48883469,
       1076.54377399, 1008.8501479 ,  879.16999577, 1113.10512979,
       1011.68599436, 1140.71103579,  942.2139917 ,  947.43099682,
        905.70827023, 1170.76756489, 1090.08978647,  940.01984683,
       1188.01937123,  992.73562723, 1008.10355555, 1190.22398542,
       1089.76797906, 1092.06247413, 1011.61485844, 1206.48257287,
       1035.20220246, 1111.66328091, 1027.07957133,  938.65645455,
        985.63342236, 1124.91207974, 1136.70178728, 1086.20163435,
       1123.75615078,  963.19983869, 1025.2854587 , 1048.30354965,
       1186.52361672,  943.7503106 , 1115.29201814,  942.4168685 ,
       1005.35095211,  985.84469094,  891.57942168, 1167.81294613,
       1215.79241419, 1136.11626704, 1130.09986349, 1029.61553

In [98]:
def progenitor_selection(population_set, fitnes_list):
    # Denna funktion väljer progenitorer (föräldrar) för nästa generation genom att använda ett sannolikhetsbaserat urval.
    # population_set är hela populationen, och fitnes_list är en lista med fitness-värden för varje lösning i populationen.

    total_fit = fitnes_list.sum()
    # Summerar alla fitness-värden i populationen. Detta används för att normalisera sannolikheterna.

    prob_list = (total_fit / fitnes_list)
    # Beräknar sannolikheten för varje lösning att bli vald som progenitor.
    # De lösningar med lägre fitness får en högre sannolikhet, eftersom fitness-värden är lägre för bättre lösningar (omvänd proportion).

    prob_list = prob_list / prob_list.sum()
    # Normaliserar sannolikheterna så att de summerar till 1, vilket krävs för att kunna använda dem i np.random.choice.

    # Notice there is the chance that a progenitor mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set), p=prob_list, replace=True)
    # Väljer en uppsättning progenitorer (föräldrar) från populationen med sannolikheterna angivna av prob_list.
    # replace=True betyder att en progenitor kan väljas flera gånger, vilket innebär att en individ kan para sig med sig själv.

    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set), p=prob_list, replace=True)
    # Samma som ovan, men för den andra uppsättningen progenitorer (andra föräldern i varje par).

    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    # Använder de valda indexen för att hämta motsvarande lösningar från population_set och skapa listor över progenitorer.

    return np.array([progenitor_list_a, progenitor_list_b])
    # Returnerar en array där progenitor_list_a och progenitor_list_b representerar föräldraparen.

progenitor_list = progenitor_selection(population_set, fitnes_list)
# Anropar funktionen för att välja progenitorer baserat på den aktuella populationen och deras fitness-värden.

progenitor_list[0][2]  # Visar den tredje progenitorn i den första progenitor-listan.


array(['Kyiv', 'Istanbul', 'Birmingham', 'Minsk', 'Bucharest', 'Warsaw',
       'Berlin', 'Munich', 'Prague', 'Rome', 'Barcelona', 'Milan',
       'Brussels', 'Moscow', 'Amsterdam', 'Sofia', 'Paris', 'Budapest',
       'Vienna', 'London'], dtype='<U10')

In [99]:
def mate_progenitors(prog_a, prog_b):
    # Denna funktion parar två progenitorer (prog_a och prog_b) och genererar en avkomma (offspring).
    
    offspring = prog_a[0:5]
    # Tar de första 5 städerna från progenitor A (prog_a) som en grund för avkomman.

    for city in prog_b:
        # Itererar över alla städer i progenitor B (prog_b).
        
        if not city in offspring:
            # Om staden från prog_b inte redan finns i offspring (ingen dubblett),
            offspring = np.concatenate((offspring, [city]))
            # Läggs staden till offspring genom att kombinera (concatenate) listan med den nya staden.

    return offspring
    # Returnerar den färdiga avkomman.

def mate_population(progenitor_list):
    # Denna funktion genererar en ny population genom att para alla progenitorpar.
    
    new_population_set = []
    # Skapar en tom lista för att lagra den nya populationen (avkommor).

    for i in range(progenitor_list.shape[1]):
        # Loopar igenom varje par av progenitorer i progenitor_list.
        
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        # Hämtar respektive progenitor (prog_a och prog_b) från progenitor_list för index i.

        offspring = mate_progenitors(prog_a, prog_b)
        # Skapar en avkomma genom att para prog_a och prog_b.

        new_population_set.append(offspring)
        # Lägger till den nya avkomman i new_population_set.

    return new_population_set
    # Returnerar den nya populationen (listan med alla avkommor).

new_population_set = mate_population(progenitor_list,)
# Anropar funktionen för att skapa en ny population genom att para progenitorerna.

new_population_set[0]
# Visar den första avkomman i den nya populationen.


array(['Minsk', 'Budapest', 'Moscow', 'Kyiv', 'London', 'Milan',
       'Bucharest', 'Paris', 'Brussels', 'Prague', 'Berlin', 'Warsaw',
       'Istanbul', 'Vienna', 'Munich', 'Rome', 'Barcelona', 'Birmingham',
       'Amsterdam', 'Sofia'], dtype='<U10')

In [100]:
def mutate_offspring(offspring):
    # Denna funktion applicerar mutation på en enskild avkomma genom att byta plats på städer i resvägen.

    for q in range(int(n_cities * mutation_rate)):
        # Loopar ett antal gånger som är baserat på antalet städer multiplicerat med mutationsfrekvensen.
        # Det anger hur många mutationer (bytesoperationer) som ska göras.

        a = np.random.randint(0, n_cities)
        b = np.random.randint(0, n_cities)
        # Väljer två slumpmässiga index inom avkommans lista över städer.

        offspring[a], offspring[b] = offspring[b], offspring[a]
        # Byter plats på städerna vid de två slumpmässigt valda indexen (a och b).

    return offspring
    # Returnerar den muterade avkomman.

def mutate_population(new_population_set):
    # Denna funktion applicerar mutation på hela den nya populationen av avkommor.

    mutated_pop = []
    # Skapar en tom lista för att lagra den muterade populationen.

    for offspring in new_population_set:
        # Loopar igenom varje avkomma i den nya populationen.
        mutated_pop.append(mutate_offspring(offspring))
        # Muterar varje avkomma med funktionen mutate_offspring och lägger till den i listan mutated_pop.

    return mutated_pop
    # Returnerar den muterade populationen.

mutated_pop = mutate_population(new_population_set)
# Anropar funktionen för att mutera den nya populationen.

mutated_pop[0]
# Visar den första muterade avkomman i den nya populationen.


array(['Vienna', 'Budapest', 'Istanbul', 'Kyiv', 'London', 'Milan',
       'Bucharest', 'Paris', 'Brussels', 'Prague', 'Berlin', 'Warsaw',
       'Moscow', 'Minsk', 'Munich', 'Rome', 'Barcelona', 'Birmingham',
       'Amsterdam', 'Sofia'], dtype='<U10')

[3123,
 619.2656888722124,
 array([['Munich', 'Moscow', 'Minsk', 'Bucharest', 'Vienna', 'Paris',
         'Prague', 'Barcelona', 'Rome', 'Warsaw', 'Amsterdam', 'Brussels',
         'Istanbul', 'London', 'Budapest', 'Berlin', 'Kyiv', 'Sofia',
         'Milan', 'Birmingham']], dtype='<U10')]